In [1]:
import pandas as pd
import numpy as np

import cufflinks as cf
import plotly.graph_objects as go
import chart_studio.plotly as py

cf.go_offline(connected = True)

from IPython.core.display import display, HTML
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf


from dash import Dash, dcc, html
import numpy as np
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px

import tensorflow as tf
import bert
import os
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tqdm import tqdm


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False



display(HTML("<style>.container { width:70% !important; }</style>"))
direct = 'C:/Users/USER/Documents/Jupyter Notebook/AI_IMEN/korpatbert_v1.0/'


import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

D:\anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3934086938663135683]

# Network

In [2]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

In [3]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [4]:
from transformers import AutoTokenizer, AutoModel#for embeddings
from tqdm import tqdm
from transformers import ElectraTokenizer, ElectraForPreTraining


direct_path = 'C:/Users/USER/Documents/Jupyter Notebook/AI_IMEN/korpatbert_v1.0/'

In [5]:
tokenizer = ElectraTokenizer.from_pretrained('Pray123/first', use_auth_token=True)
model = ElectraForPreTraining.from_pretrained('Pray123/first', use_auth_token=True)

In [6]:
tqdm.pandas()

In [7]:
df1 = pd.read_csv(direct_path + 'cs_new_total.csv',encoding = 'utf-8-sig', index_col = 0)

In [8]:
df1.columns

Index(['제목', '출원인', '피인용 횟수', 'IPC', '출원번호(일자)', '최종권리자', '요약', '(51) Int. CL',
       '(52) CPC ', '(21) 출원번호/일자(국제)', '(71) 출원인', '번역문제출일자', '(11) 등록번호/일자',
       '(65) 공개번호/일자', '(11) 공고번호/일자', '(86) 국제출원번호/일자', '(87) 국제공개번호/일자',
       '(30) 우선권정보', '법적상태', '심사진행상태', '심판사항', '구분', '원출원번호/일자(국제)', '관련 출원번호',
       '기술이전 희망', '심사청구여부/일자', '심사청구항수', '(21) 출원번호/일자', '원출원번호/일자',
       'cosing_simliarity'],
      dtype='object')

In [9]:
df1['요약_tokenize'] = ''

In [10]:
def make_tokenize(row):
    w = row['요약']
    ww = tokenizer(w,max_length = 512)['input_ids']
    row['요약_tokenize'] = ww
    
    return row

In [11]:
df2 = df1.progress_apply(make_tokenize,axis=1).copy()

100%|████████████████████████████████████████████████████████████████████████████| 30039/30039 [05:50<00:00, 85.75it/s]


In [12]:
import re

def make_sep(df):
    regex = "\(.*\)|\s-\s.*"
    lst_all = []
    for idx, CPC in zip(df['(52) CPC '].index, df['(52) CPC ']):
        a = CPC.replace("'",'').replace('[','').replace(']','').split(',')
        lst_small = []
        for ww in a:
            ww_sub = re.sub(regex, '', ww.strip())
            ww_sub_split = ww_sub.split('/')
            lst_small.append(ww_sub_split[0][0:4].strip() + ' ' + ww_sub_split[0][4:6].strip())
            
        lst_all.append(lst_small)
        
    return lst_all

In [20]:
values = []
for i in df2['요약_tokenize'].values:
    values += i

In [21]:
values_요약 = pd.Series(values).value_counts()

In [22]:
values_요약

4681     575329
2780     574630
2620     537376
7        511404
2535     487693
          ...  
29442         1
9471          1
24534         1
18447         1
27281         1
Length: 22864, dtype: int64

In [23]:
index_요약 = values_요약[((values_요약.values  <= values_요약.quantile(0.95)) & (values_요약.values >  values_요약.quantile(0.05)))].index # 앞단 5 % 자름

In [24]:
index_요약

Int64Index([ 6854, 14250,    53, 27951,  2957,  6829,  5956,  5254, 14762,
             1528,
            ...
            19926, 22886, 30673, 34172, 29542, 13959, 18219, 23728,  6177,
             5226],
           dtype='int64', length=19772)

In [25]:
pretreind_model_dir = 'C:/Users/USER/Documents/Jupyter Notebook/AI_IMEN/korpatbert_v1.0/pretrained'

In [32]:
input_ids = keras.layers.Input(shape=(512,), dtype='int32')
# BERT 언어모델 레이어 생성
bert_params = bert.params_from_pretrained_ckpt(pretreind_model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

bert_output = l_bert(input_ids)
print("bert shape", bert_output.shape)

bert shape (None, 512, 768)


In [43]:
vector = l_bert.embeddings_layer.word_embeddings_layer.variables[0].numpy()


In [44]:
index_요약

Int64Index([ 6854, 14250,    53, 27951,  2957,  6829,  5956,  5254, 14762,
             1528,
            ...
            19926, 22886, 30673, 34172, 29542, 13959, 18219, 23728,  6177,
             5226],
           dtype='int64', length=19772)

In [45]:
standard = """본 발명은 블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 관한 것으로, 이 시스템은, 상기 프랜차이즈 유통 및 관리 시스템은 프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고, 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고, 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 물류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저장하는 거래내역 등재 모듈을 포함하고, 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고, 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된다
블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 있어서,

프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고,

상기 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고,

상기 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 문류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저자하는 거래내역 등재 모듈을 포함하고,

상기 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고,

상기 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된 것을 특징으로 하는

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템.

제1항에 있어서,

가맹점측 단말로부터 중개관리 서버를 통해 본점측 단말에 물품 구매 요청이 발생되면, 중개관리 서버는 금융사측 단말에 가상계좌 개설을 요청하고, 중개관리 서버가 금융사측 단말로부터 가성계좌 개설을 통보받으면 개설된 가상계좌를 가맹점측 단말에 전송하도록 구성된

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템.

제2항에 있어서,

가맹점측 단말에서 개설된 가상계좌에 입금을 수행하면 금융사측 단말은 입금 확인을 중개 관리 서버에 전송하고, 중개 관리 서버는 본점측 단말에 금융사측으로부터의 입금 확인을 전달한 후, 입금 거래 내역을 블록체인 주노드에 저장하고 주노드에 저장된 입급 거래 내역을 금융사측 단말의 블록체인 부노드, 가맹점측 단말의 블록체인 부노드, 본점측 단말의 블록체인 부노드에 복제하여 분산 저장하도록 구성된 것을 특징으로 하는

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템.

제1항에 있어서,

중개 관리 서버는 상품의 수요를 예측하기 위한 수요 예측 모듈을 더 포함하고, 상기 수요 예측 모듈은,

가맹점측 단말로부터 월, 일자, 요일, 상품 정보를 포함하는 판매 상품 데이터를 수집하는 데이터 수집부;

수집된 판매 상품 데이터를 월별, 일자별, 요일별 판매 상품 항목으로 분류하는 데이터 분류부;

분류된 데이터를 분석 정의 데이터에 부합하여 분석하여 분석 데이터를 생성하는 데이터 분석부 - 분석 정의 데이터는 분석실행 방식, 분석수행 주기, 분석대상 평가항목 또는 가중치 중에 어느 하나를 포함하며, 분석 데이터는 판매 품목 대한 빈도수를 월별, 요일별, 일자별, 시간대별 수치화한 데이터를 포함하여 구성됨 - ;

분석 데이터를 딥러닝을 통한 머신러닝을 수행하는 데이터 학습부; 및

데이터 학습부로부터 인가받은 머신러닝 결과에 기반하여 월별, 일자별, 요일별, 시간대별, 품목 판매 예측값을 제공하는 수요 예측부를 포함하는

블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템."""

In [46]:
standard_2 = standard.replace('\n',' ')

In [47]:
from korpat_tokenizer import Tokenizer

tokenizer = Tokenizer(vocab_path="korpat_vocab.txt", cased=True)

In [48]:
standard_2_token = tokenizer.tokenize(standard_2)

In [76]:
standard_2_token

['[CLS]',
 '본',
 '발명',
 '은',
 '블록',
 '##체인',
 '및',
 '머신',
 '##러',
 '##닝',
 '을',
 '이용',
 '한',
 '프',
 '##랜',
 '##차',
 '##이즈',
 '유통',
 '및',
 '관리',
 '시스템',
 '에',
 '관한',
 '것',
 '으로',
 ',',
 '이',
 '시스템',
 '은',
 ',',
 '상기',
 '프',
 '##랜',
 '##차',
 '##이즈',
 '유통',
 '및',
 '관리',
 '시스템',
 '은',
 '프',
 '##랜',
 '##차',
 '##이즈',
 '협력',
 '##업체',
 '단말',
 '과',
 '통신',
 '가능',
 '하',
 '도록',
 '구성',
 '된',
 '중개',
 '관리',
 '서버',
 '를',
 '포함',
 '하',
 '고',
 ',',
 '협력',
 '##업체',
 '단말',
 '은',
 '프',
 '##랜',
 '##차',
 '##이즈',
 '본',
 '##점',
 '측',
 '단말',
 ',',
 '가맹점',
 '##측',
 '단말',
 ',',
 '물류',
 '##측',
 '단말',
 ',',
 '및',
 '금융',
 '##사',
 '##측',
 '단말',
 '을',
 '포함',
 '하',
 '고',
 ',',
 '중개',
 '관리',
 '서버',
 '는',
 '상품',
 '의',
 '주문',
 '을',
 '관리',
 '하',
 '는',
 '주문',
 '##관리',
 '를',
 '수행',
 '하',
 '는',
 '주문',
 '##관리',
 '모듈',
 ',',
 '물류',
 '를',
 '관리',
 '하',
 '는',
 '수',
 '행하',
 '는',
 '물류',
 '##관리',
 '모듈',
 ',',
 '구매',
 '/',
 '생산',
 '관리',
 '를',
 '수행',
 '하',
 '는',
 '구매',
 '/',
 '생산',
 '##관리',
 '모듈',
 ',',
 '재고',
 '/',
 '손',
 '##익',
 '을',


In [51]:
log_dir = 'C:/Users/USER/Documents/Jupyter Notebook/AI_IMEN/korpatbert_v1.0/pretrained'

In [52]:
lst_vocab = []
with open(log_dir + '/korpat_vocab.txt',encoding='utf-8-sig') as f:
    for idx,i in enumerate(f.readlines()):
        
        ww= re.sub(r"[^가-힣]", "", i)
        
        
        if ('#' in ww):
            pass
        elif len(ww.replace('\n','')) <= 1:
            pass
        
        else:
            
            lst_vocab.append([idx,ww.replace('\n','')])
        
    f.close()

In [53]:
lst_vocab_standard = []

for idx,i in enumerate(standard_2_token):

    ww= re.sub(r"[^가-힣]", "", i)


    if ('#' in ww):
        pass
    elif len(ww.replace('\n','')) <= 1:
        pass
    else:
        lst_vocab_standard.append(ww.replace('\n',''))



In [78]:
len(lst_vocab_standard)

531

In [57]:
lst_union = []
for i in lst_vocab_standard:
    for j in lst_vocab:
        if j[1] == i:
            lst_union.append(j[0])
            break
            
            
lst_union

[77,
 500,
 3118,
 4685,
 184,
 2520,
 4977,
 682,
 251,
 439,
 33,
 251,
 23,
 2520,
 4977,
 682,
 251,
 2520,
 7588,
 6991,
 337,
 382,
 236,
 118,
 144,
 7287,
 682,
 533,
 79,
 7588,
 6991,
 337,
 2520,
 337,
 3854,
 337,
 10492,
 337,
 1506,
 337,
 79,
 7287,
 682,
 533,
 1316,
 4062,
 682,
 4062,
 682,
 353,
 4062,
 682,
 486,
 10492,
 682,
 1500,
 10492,
 682,
 486,
 2287,
 1117,
 682,
 353,
 2287,
 1117,
 682,
 486,
 11637,
 682,
 11637,
 682,
 486,
 2809,
 6860,
 500,
 3118,
 687,
 301,
 2809,
 6860,
 486,
 79,
 500,
 3118,
 687,
 2809,
 6860,
 301,
 500,
 3118,
 926,
 687,
 500,
 3118,
 926,
 687,
 301,
 2809,
 6860,
 282,
 33,
 1510,
 301,
 500,
 3118,
 926,
 687,
 79,
 7287,
 682,
 533,
 500,
 3118,
 926,
 687,
 185,
 23,
 7588,
 6991,
 337,
 500,
 3118,
 926,
 687,
 185,
 7287,
 682,
 533,
 500,
 3118,
 7588,
 6991,
 226,
 2809,
 274,
 926,
 687,
 2809,
 6860,
 301,
 735,
 926,
 687,
 2809,
 6860,
 5085,
 1051,
 301,
 118,
 144,
 93,
 500,
 3118,
 4685,
 184,
 2520,
 4977,

In [77]:
df_vocab

,0
0,상기
1,으로
2,에서
3,한다
4,또는
...,...
14830,유모
14831,리도
14832,장벽
14833,해석


In [68]:
vector_2 = pd.DataFrame(vector[[i[0] for i in lst_vocab]])

In [72]:
df_vocab = pd.DataFrame([[i[1] for i in lst_vocab]]).T.copy()

In [74]:
# df_vocab.to_csv('metadata_2.tsv', sep="\t",index = False, header = None)

In [75]:
# vector_2.to_csv('vector_2.tsv', sep="\t",index = False, header = None)

In [59]:
vector.shape

(21400, 768)

In [ ]:
vector

In [60]:
vector_standard = pd.DataFrame(vector).iloc[lst_union]

In [61]:
len(standard_2_token)

1085

In [62]:
vector_standard

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
77,0.040401,0.023655,0.049670,-0.026273,-0.045711,0.023039,0.018300,-0.011622,-0.040082,0.000504,...,0.018928,0.042244,-0.031087,-0.023898,0.042254,0.025068,0.033710,0.030678,0.023913,0.022275
500,-0.029742,-0.032787,0.002594,0.035262,0.044066,0.033617,0.047810,0.032708,-0.040630,0.038394,...,-0.044878,0.001985,0.003326,-0.010265,-0.011167,-0.002417,-0.028098,-0.003610,-0.018011,-0.034172
3118,-0.034360,-0.018917,-0.032074,-0.023723,-0.007835,0.042305,-0.018014,0.027528,-0.017057,-0.037545,...,0.044725,-0.009602,0.043437,-0.014738,-0.047027,-0.018624,0.044934,0.002775,0.030342,-0.008744
4685,-0.033909,-0.032984,-0.039282,0.044671,-0.019392,-0.028205,-0.002746,0.028654,-0.004459,0.013464,...,-0.024501,-0.038891,0.003239,0.020706,-0.003559,0.029144,0.041612,-0.029718,-0.023080,-0.039635
184,-0.038650,-0.036079,-0.028822,-0.003033,0.034272,-0.044268,0.026905,0.043830,0.047009,0.042179,...,0.034025,-0.017803,0.011988,-0.000184,-0.013127,0.022989,-0.002486,0.035219,0.017723,-0.013448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,-0.038650,-0.036079,-0.028822,-0.003033,0.034272,-0.044268,0.026905,0.043830,0.047009,0.042179,...,0.034025,-0.017803,0.011988,-0.000184,-0.013127,0.022989,-0.002486,0.035219,0.017723,-0.013448
2520,0.007568,0.024767,-0.041780,0.020578,0.016925,-0.000796,-0.042688,-0.009925,0.043480,-0.035523,...,0.034055,-0.025146,0.043266,0.003305,0.037046,-0.020669,-0.044730,-0.038690,0.048529,0.045416
4977,-0.048116,0.001193,0.018074,0.041857,-0.016721,0.022072,0.030648,0.036816,-0.026944,0.015687,...,-0.027425,0.045851,-0.022467,0.003118,0.013128,-0.047554,-0.021253,0.025568,0.028283,-0.003191
682,-0.033043,0.022891,0.014905,0.030300,0.046381,0.041029,0.014401,0.000851,-0.019854,0.012156,...,-0.035277,-0.025050,-0.021194,0.007098,-0.028775,-0.000718,0.030157,-0.023466,0.018546,-0.030288


In [79]:
df_vocab_standard = pd.DataFrame(lst_vocab_standard).copy()

In [85]:
# pd.DataFrame(df_vocab_standard.value_counts()).to_csv('standard.csv',encoding = 'utf-8-sig')

In [ ]:
df_vocab_standard_2 = df_vocab_standard.drop_duplicates().copy()

In [ ]:
df_vocab_standard_2.index

In [ ]:
vector_standard_2 = vector_standard.reset_index(drop = True).iloc[df_vocab_standard_2.index].copy()

In [ ]:
df_vocab_standard

In [ ]:
df_vocab_standard_2.to_csv('metadata_standard.tsv', sep="\t",index = False, header = None)

In [ ]:

vector_standard_2.to_csv('vector_standard.tsv', sep="\t",index = False, header = None)

In [ ]:
vector_2

In [20]:
pd.read_csv('metadata_standard.tsv', sep="\t")

,발명
0,블록
1,체인
2,머신
3,이용
4,이즈
...,...
97,결과
98,기반
99,예측값
100,제공


In [ ]:
from flask import Flask

app = Flask(__name__, static_url_path='/static')

@app.route('/pandas')
def make_read_excel():
    ## 반드시 static에 있지 않아도 읽을 수는 있음.
    ## 현재 파일과 읽으려는 파일이 같은 경로에 있기 때문에 아래와 같은 방식으로 읽을 수도 있음.
    import pandas as pd 
    import numpy as np 

    ## make excel file 
    writer = pd.ExcelWriter('static/excel_for_flask.xlsx')
    df = pd.DataFrame({"col_{}".format(i):list(np.random.randint(0, 100, 100)) for i in range(0, 8)})
    df.to_excel(writer, 'sheet1')
    writer.save()

    ## read excel file 
    df = vector_standard_2
    ## 아주 다행히도, dataframe을 html 문서로 바로 변환해주는 형식이 있습니다. 
    return df.to_html()

if  __name__ == '__main__':
    app.run(port=8000)

In [ ]:
vector

# word cloud


In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px